# Set Up/Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# conda activate "/pscratch/sd/m/mansisak/memorization/env/"

In [3]:
# !pip list

In [4]:
#pip install accelerate

In [5]:
import torch
import os
from transformers import AutoModelForCausalLM
from torch.utils.data import DataLoader
from tqdm import tqdm

### TODO: figure out automatically
# device = "cpu"
device = "cuda:1"

/global/cfs/cdirs/m636/geniesse/envs/memorization/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


# Load in Models

In [6]:
def print_edited_model_paths(parent_path):
    
    total_exp = 0
    for model_name in ['pythia-6.9b-deduped', 'pythia-2.8b-deduped']:
        y_idx = 0
        for step in [36000, 72000, 108000, 143000]:
            for loc_method in [
                  "act",
                  "hc",
                  "slim",
                  "durable",
                  "durable_agg",
                  "random",
                  "random_greedy",
                  "greedy",
              ]:
                
                for ratio in [0.00001, 0.0001, 0.001, 0.01, 0.05, 0.1, 0.25, 0.3]:
                    result_path = f"{parent_path}{step}/EleutherAI_edit/{loc_method}/mem/{ratio}"
                    if loc_method not in ["random", "random_greedy"]:
                        if ratio >= 0.1:
                            continue

                    # this ratio is too small for neuron-level methods
                    if loc_method in ["zero", "hc", "ig", "slim", "act"]:
                        if ratio <= 0.0001:
                            continue

                    if loc_method in ["greedy"]:
                        if ratio > 0.00001:
                            continue
                            
                    ######
                    if loc_method in ["greedy", "durable", "durable_agg","act"]:
                        model_path = f'{result_path}/{model_name}'
                        total_exp += 1

                    if loc_method in ["slim", "hc"]:
                        for epochs in [1, 10, 20]:
                            total_exp += 1
                            model_path = f'{result_path}/{epochs}/1000/0.1/0.1/{model_name}'                      

                    if loc_method in ["random", "random_greedy"]:
                        for epochs in [1, 10, 20]:
                            total_exp += 1
                            model_path = f'{result_path}/{epochs}/0.1/0.9/0.0005/{model_name}'
                    if os.path.isfile(model_path):
                        print("edited model exists:", model_path)
                    else:
                        print("edited model doesn't exist yet: ", model_path)

    print("total_expeirments: ", total_exp)



In [7]:
# parent_path = "/pscratch/sd/m/mansisak/memorization/model_ckpts/"
parent_path = "/pscratch/sd/m/mansisak/memorization/model_ckpts/"

print_edited_model_paths(parent_path)

edited model exists: /pscratch/sd/m/mansisak/memorization/model_ckpts/36000/EleutherAI_edit/act/mem/0.001/pythia-6.9b-deduped
edited model exists: /pscratch/sd/m/mansisak/memorization/model_ckpts/36000/EleutherAI_edit/act/mem/0.01/pythia-6.9b-deduped
edited model exists: /pscratch/sd/m/mansisak/memorization/model_ckpts/36000/EleutherAI_edit/act/mem/0.05/pythia-6.9b-deduped
edited model exists: /pscratch/sd/m/mansisak/memorization/model_ckpts/36000/EleutherAI_edit/hc/mem/0.001/20/1000/0.1/0.1/pythia-6.9b-deduped
edited model exists: /pscratch/sd/m/mansisak/memorization/model_ckpts/36000/EleutherAI_edit/hc/mem/0.01/20/1000/0.1/0.1/pythia-6.9b-deduped
edited model exists: /pscratch/sd/m/mansisak/memorization/model_ckpts/36000/EleutherAI_edit/hc/mem/0.05/20/1000/0.1/0.1/pythia-6.9b-deduped
edited model exists: /pscratch/sd/m/mansisak/memorization/model_ckpts/36000/EleutherAI_edit/slim/mem/0.001/20/1000/0.1/0.1/pythia-6.9b-deduped
edited model exists: /pscratch/sd/m/mansisak/memorization/mo

In [8]:
# Example of how to load in a model:

model = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/pythia-2.8b-deduped",
    torch_dtype=torch.float16,
    device_map=device
)

# sd_path = '/pscratch/sd/m/mansisak/memorization/model_ckpts/old_run/143000/EleutherAI_edit/random_greedy/mem/0.001/20/0.1/0.9/0.0005/pythia-2.8b-deduped'
sd_paths = dict(
    unedited = None,
    good_edit = "/pscratch/sd/m/mansisak/memorization/model_ckpts/143000/EleutherAI_edit/random_greedy/mem/0.001/20/0.1/0.9/0.0005/pythia-2.8b-deduped",
    bad_edit = "/pscratch/sd/m/mansisak/memorization/model_ckpts/143000/EleutherAI_edit/random/mem/0.1/20/0.1/0.9/0.0005/pythia-2.8b-deduped",
)

# choose which model to load
use_model = "bad_edit"
sd_path = sd_paths.get(use_model)

if sd_path is not None:
    sd = torch.load(sd_path, map_location=device)["model_state_dict"]
    if "random" in sd_path:
        for k in sd:
            if "4h" in k:
                #rint(sd[k].shape)
                #rint(k)
                sd[k] = sd[k].T
    model.load_state_dict(sd, assign=True)
model.eval()

/tmp/ipykernel_1964286/3153447877.py:27: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3637.)
  sd[k] = sd[k].T


GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 2560)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
          (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=True)


In [9]:
!ls -ls /pscratch/sd/m/mansisak/memorization/model_ckpts/143000

total 4
4 drwxr-xr-x 11 mansisak mansisak 4096 Sep  3 06:26 EleutherAI_edit


In [10]:
model.device

device(type='cuda', index=1)

# Load in Data

In [11]:
random_data = torch.load("/pscratch/sd/m/mansisak/memorization/src/data/pythia_mem_data/pile_random_batch.pt")
random_data_pile = torch.reshape(random_data[0:2040], (3264, 80)).to(device)

random_data = random_data_pile[0:1632]
random_subset_data = random_data_pile[0:10] # for testing purposes
extra_data = random_data_pile[1632:]
random_dataloader = DataLoader(random_data, batch_size=1, shuffle=False)
random_subset_dataloader = DataLoader(random_subset_data, batch_size=1, shuffle=False)
random_dataloader

In [12]:
# need (inputs, targets) format for PyHessian
hessian_dataloader = DataLoader([*zip(random_data, random_data)], batch_size=1, shuffle=False)
hessian_dataloader

In [13]:
# need (inputs, targets) format for PyHessian
hessian_subset_dataloader = DataLoader([*zip(random_subset_data, random_subset_data)], batch_size=1, shuffle=False)
hessian_subset_dataloader

# Model Inference

In [14]:
def perplexity(model, dataloader):
    avg_metric = 0
    for i, batch in enumerate(dataloader):
        with torch.no_grad():
            model_output = model(batch, labels=batch)
        loss = model_output.loss
        loss_exp = torch.exp(loss)
        if torch.isinf(loss_exp):
            print(f"Skipping invalid loss...")
            print(f"    {i=}, loss={loss.item()}, exp(loss)={torch.exp(loss).item()}")
        avg_metric += loss_exp
        # print(loss.item())
    return avg_metric / len(dataloader)


def perplexity_criterion(model_output, targets=None):
    loss = model_output.loss
    return torch.exp(loss)


def average_loss(model, dataloader, num_batches=-1):
    avg_metric = 0
    for i, batch in enumerate(dataloader):
        if (num_batches > 0) and (i >= num_batches):
            break
        with torch.no_grad():
            model_output = model(batch, labels=batch)
        loss = model_output.loss
        avg_metric += loss
        # print(loss.item())
    return avg_metric / len(dataloader)


from torch.nn import CrossEntropyLoss
def loss_criterion(model_output, inputs):
    # Shift so that tokens < n predict n
    shift_labels = inputs[..., 1:].contiguous()
    shift_logits = model_output.logits[..., :-1, :].contiguous()
    # Calculate per-token loss
    loss_fct = CrossEntropyLoss(reduction="none")
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    # Resize and average loss per sample
    loss_per_sample = loss.view(shift_logits.size(0), shift_logits.size(1)).mean(axis=1)
    return (loss_per_sample).mean()


In [ ]:
average_loss(model, random_dataloader)

In [ ]:
loss_criterion(model(random_data[:10]), random_data[:10])

# Loss Landscape stuff

In [15]:
import numpy as np
from pyhessian import hessian
from pyhessian.utils import normalization
import copy

import matplotlib.pyplot as plt

In [16]:
def get_params(model_orig, model_perb, direction, alpha):
    for m_orig, m_perb, d in zip(
        model_orig.parameters(), model_perb.parameters(), direction
    ):
        m_perb.data = m_orig.data + alpha * d
    return model_perb

## 2D Hessian directions

In [29]:

import torch 
from torch.autograd import grad
from scipy.sparse.linalg import LinearOperator, eigsh
import numpy as np

def get_hessian_eigenvectors(model_, compute_loss_fn, train_data_loader, num_batches, device, n_top_vectors, param_extract_fn):
    """
    model: a pytorch model
    loss_fn: a pytorch loss function
    train_data_loader: a pytorch data loader
    num_batches: number of batches to use for the hessian calculation
    device: the device to use for the hessian calculation
    n_top_vectors: number of top eigenvalues / eigenvectors to return
    param_extract_fn: a function that takes a model and returns a list of parameters to compute the hessian with respect to (pass None to use all parameters)
    returns: a tuple of (eigenvalues, eigenvectors)
    eigenvalues: a numpy array of the top eigenvalues, arranged in increasing order
    eigenvectors: a numpy array of the top eigenvectors, arranged in increasing order, shape (n_top_vectors, num_params)
    """
    param_extract_fn = param_extract_fn or (lambda x: x.parameters())
    num_params = sum(p.numel() for p in param_extract_fn(model_))
    # subset_images, subset_labels = [], []
    # for batch_idx, (images, labels) in enumerate(train_data_loader):
    #     if batch_idx >= num_batches:
    #         break
    #     subset_images.append(images.to(device))
    #     subset_labels.append(labels.to(device))
    # subset_images = torch.cat(subset_images)
    # subset_labels = torch.cat(subset_labels)
    
    def hessian_vector_product(vector):
        model_.zero_grad()
        # grad_params = grad(compute_loss_fn(model, subset_images, subset_labels), param_extract_fn(model), create_graph=True)
        grad_params = grad(compute_loss_fn(model_, train_data_loader, num_batches), model_.parameters(), create_graph=True)
        flat_grad = torch.cat([g.view(-1) for g in grad_params])
        grad_vector_product = torch.sum(flat_grad * vector)
        hvp = grad(grad_vector_product, model_.parameters(), retain_graph=True)
        return torch.cat([g.contiguous().view(-1) for g in hvp])

    def matvec(v):
        v_tensor = torch.tensor(v, dtype=torch.float32, device=device)
        return hessian_vector_product(v_tensor).cpu().detach().numpy()

    linear_operator = LinearOperator((num_params, num_params), matvec=matvec)
    eigenvalues, eigenvectors = eigsh(linear_operator, k=n_top_vectors, tol=0.001, which='LM', return_eigenvectors=True)
    eigenvectors = np.transpose(eigenvectors)
    return eigenvalues, eigenvectors
  

In [30]:

def average_loss(model, dataloader, num_batches=-1):
    avg_metric = 0
    for i, batch in enumerate(dataloader):
        if (num_batches > 0) and (i >= num_batches):
            break
        with torch.no_grad():
            model_output = model(batch, labels=batch)
        loss = model_output.loss
        avg_metric += loss
        # print(loss.item())
        
    return avg_metric / len(dataloader)


In [31]:
device = "cuda:1"

# with torch.no_grad():
# model.eval()
# model_perb = copy.deepcopy(model)
# model_perb.eval()

# model = model.to("cpu")
# model_perb = model_perb.to(device)

In [32]:
model.train()

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 2560)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
          (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=True)


In [33]:
for module in model.modules():
    for p in module.parameters():
        p.requires_grad = True

In [34]:

### Compute top Hessian eigenvectors
# [version 1] compute eigenvectors
top_eigenvalues, top_eigenvectors = get_hessian_eigenvectors(
    model, average_loss, copy.deepcopy(random_subset_dataloader), -1, 'cuda:1', 2, None
)
top_eigenvalues = top_eigenvalues[::-1]
top_eigenvectors = top_eigenvectors[::-1, :]
print(f"            [version 1] {top_eigenvalues=}")


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn